In [239]:
import pandas as pd
import numpy as np

In [240]:
indexCol = 'i'
outcomeCol = 'Y'
covariateCol = 'X'
treatmentCol = 'treatment'

df = pd.read_csv("C:/Users/17172/Desktop/CausalLearning/sample_data/test.txt")
df.set_index([indexCol])

,treatment,X,Y
i,,,
1,0,2,7
2,0,4,8
3,0,5,6
4,1,3,9
5,1,2,8
6,1,3,6
7,1,1,5


In [241]:
emptyVal = [0] * df.index
df.insert(len(df.columns), 'matches', emptyVal)
df.set_index([indexCol])

,treatment,X,Y,matches
i,,,,
1,0,2,7,0
2,0,4,8,0
3,0,5,6,0
4,1,3,9,0
5,1,2,8,0
6,1,3,6,0
7,1,1,5,0


In [242]:
for i in range(len(df)):
    base = df.iloc[i]
    dfOfMatches = df.iloc[(df[covariateCol]-base[covariateCol]).abs().argsort()[:]]
    dfOfMatches = dfOfMatches[dfOfMatches[treatmentCol] != base[treatmentCol]]
    temp = abs((int(dfOfMatches.iloc[0][covariateCol])-int(base[covariateCol])))

    listOfMatches = []

    searchVal = int(base[covariateCol])
    covariateVal = df[covariateCol]
    queryResult = dfOfMatches.query('@covariateVal-@searchVal == @temp | @searchVal-@covariateVal == @temp')['i']
    for x in queryResult:
        listOfMatches.append(int(x))

    finalMatches = str(listOfMatches).replace('[', '')
    finalMatches = finalMatches.replace(']', '')

    df.at[i, 'matches'] = finalMatches
df


,i,treatment,X,Y,matches
0,1,0,2,7,5
1,2,0,4,8,"4, 6"
2,3,0,5,6,"4, 6"
3,4,1,3,9,"1, 2"
4,5,1,2,8,1
5,6,1,3,6,"1, 2"
6,7,1,1,5,1


In [243]:
total = 0
for i in range(len(df)):    
    treat = int(df.iloc[i][treatmentCol])
    outcomeValue = int(df.iloc[i]['Y'])
    indecovariateColMatches = df.iloc[i]['matches'].split(",")
    indecovariateColMatches = [eval(j) for j in indecovariateColMatches]

    outcomeMatch = df.loc[(df[indexCol].isin(indecovariateColMatches))][outcomeCol].mean()

    if treat == 0:
        finalOutcome = outcomeMatch - outcomeValue
    else:
        finalOutcome = outcomeValue - outcomeMatch
    total = total + finalOutcome

total/len(df)

0.14285714285714285